In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras_preprocessing

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install TensorFlow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import PIL  
from keras.models import model_from_json
import tensorflow as tf

In [11]:
from warnings import filterwarnings
filterwarnings('ignore')

In [12]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [13]:

TEST_DIR = 'images/test'
TRAIN_DIR = 'images/train'

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        label_dir = os.path.join(dir, label)
        if os.path.isdir(label_dir):  # Dizin olup olmadığını kontrol edin
            for imagename in os.listdir(label_dir):
                if not imagename.startswith('.'):
                    image_path = os.path.join(label_dir, imagename)
                    if os.path.isfile(image_path):  # Dosya olup olmadığını kontrol edin
                        image_paths.append(image_path)
                        labels.append(label)
            print(label, "tamamlandı")
    return image_paths, labels




In [14]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

happy tamamlandı
sad tamamlandı
fear tamamlandı
surprise tamamlandı
neutral tamamlandı
angry tamamlandı
disgust tamamlandı


In [15]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

happy tamamlandı
sad tamamlandı
fear tamamlandı
surprise tamamlandı
neutral tamamlandı
angry tamamlandı
disgust tamamlandı


In [16]:
test['image']

0         images/test/happy/23933.jpg
1         images/test/happy/24906.jpg
2         images/test/happy/18033.jpg
3         images/test/happy/15271.jpg
4         images/test/happy/26888.jpg
                    ...              
7061    images/test/disgust/20761.jpg
7062    images/test/disgust/28710.jpg
7063    images/test/disgust/23876.jpg
7064     images/test/disgust/9460.jpg
7065    images/test/disgust/35580.jpg
Name: image, Length: 7066, dtype: object

In [17]:
def extract_features(images):
    
    features=[]
    
    for image in tqdm(images):
        
        img = load_img(image,grayscale=True)
        img=np.array(img)
        features.append(img)
        
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    
    return features
    

In [18]:
train.head()

,image,label
0,images/train/happy/3578.jpg,happy
1,images/train/happy/16988.jpg,happy
2,images/train/happy/2666.jpg,happy
3,images/train/happy/5109.jpg,happy
4,images/train/happy/11981.jpg,happy


In [19]:
train_feature=extract_features(train['image'])

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [20]:
test_feature=extract_features(test['image'])

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
x_train=train_feature/255.0
x_test=test_feature/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [ ]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [ ]:
model=Sequential()

# Convolutional layers

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
## fully connected Layers

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(7,activation='softmax'))



In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [9]:
model_json_path = "emotiondetector.json"  # Modelin .json dosyasının yolu
with open(model_json_path, "r") as json_file:
    loaded_model_json = json_file.read()
    loaded_model = tf.keras.models.model_from_json(loaded_model_json)
    
    
# Modelin ağırlıklarını yükleyin (örnek olarak HDF5 format kullanılıyor)
model_weights_path = "emotiondetector.h5"  # Modelin ağırlıklarının olduğu dosyanın yolu
loaded_model.load_weights(model_weights_path)

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:

image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')